In [1]:
import cv2
import numpy as np
import functools
import operator
import os

TARGET_WIDTH = 128
TARGET_HEIGHT = 128
# module level variables ##########################################################################
MIN_CONTOUR_AREA = 100

RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

In [2]:
class ContourWithData():

    # member variables ############################################################################
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    def calculateRectTopLeftPointAndWidthAndHeight(self):               # calculate bounding rect info
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    def checkIfContourIsValid(self):                            # this is oversimplified, for a production grade program
        if self.fltArea < MIN_CONTOUR_AREA: return False        # much better validity checking would be necessary
        return True

In [3]:
allContoursWithData = []                # declare empty lists,
validContoursWithData = []              # we will fill these shortly

try:
    npaClassifications = np.loadtxt("classifications1.txt", np.float32)                  # read in training classifications
except:
    print("error, unable to open classifications.txt, exiting program\n")
    os.system("pause")
# end try

try:
    npaFlattenedImages = np.loadtxt("flattened_images1.txt", np.float32)                 # read in training images
except:
    print("error, unable to open flattened_images.txt, exiting program\n")
    os.system("pause")
    # end try

In [4]:
# reshape numpy array to 1d, necessary to pass to call to train
npaClassifications = npaClassifications.reshape((npaClassifications.size, 1))

In [5]:
kNearest = cv2.ml.KNearest_create()

In [6]:
kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)

True

In [7]:
img = cv2.imread('a\plateFront1.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [8]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 4)
cv2.imshow("thresh", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()
#ret, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)

In [9]:
_, labels = cv2.connectedComponents(thresh)
mask = np.zeros(thresh.shape, dtype="uint8")

# cv2.imshow("thresh", labels)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
total_pixels = img.shape[0] * img.shape[1]
# lower = total_pixels // 50 # heuristic param, can be fine tuned if necessary
# upper = total_pixels // 5 # heuristic param, can be fine tuned if necessary

lower = total_pixels // 100 # heuristic param, can be fine tuned if necessary
upper = total_pixels // 5 # heuristic param, can be fine tuned if necessary

In [11]:
# Loop over the unique components
for (i, label) in enumerate(np.unique(labels)):
    # If this is the background label, ignore it
    if label == 0:
        continue
 
    # Otherwise, construct the label mask to display only connected component
    # for the current label
    labelMask = np.zeros(thresh.shape, dtype="uint8")
    labelMask[labels == label] = 255
    numPixels = cv2.countNonZero(labelMask)
 
    # If the number of pixels in the component is between lower bound and upper bound, 
    # add it to our mask
    if numPixels > lower and numPixels < upper:
        mask = cv2.add(mask, labelMask)
cv2.imshow("thresh", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()        

In [12]:
maskCopy = mask.copy()

In [13]:
npaContours, npaHierarchy = cv2.findContours(maskCopy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [14]:
for npaContour in npaContours:                             # for each contour
    contourWithData = ContourWithData()                                             # instantiate a contour with data object
    contourWithData.npaContour = npaContour                                         # assign contour to contour with data
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)     # get the bounding rect
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()                    # get bounding rect info
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)           # calculate the contour area
    allContoursWithData.append(contourWithData)          

In [15]:
for contourWithData in allContoursWithData:                 # for all contours
    if contourWithData.checkIfContourIsValid():             # check if valid
        validContoursWithData.append(contourWithData) 

In [16]:
# sort contours from left to right
validContoursWithData.sort(key = operator.attrgetter("intRectX"))

In [17]:
# final number sequence by the end of the program
strFinalString = ""

In [18]:
 for contourWithData in validContoursWithData:            # for each contour
                                                # draw a green rect around the current char
    cv2.rectangle(img,                                        # draw rectangle on original testing image
                      (contourWithData.intRectX, contourWithData.intRectY),     # upper left corner
                      (contourWithData.intRectX + contourWithData.intRectWidth, contourWithData.intRectY + contourWithData.intRectHeight),      # lower right corner
                      (0, 255, 0),              # green
                      2)                        # thickness

    imgROI = maskCopy[contourWithData.intRectY : contourWithData.intRectY + contourWithData.intRectHeight,     # crop char out of threshold image
                           contourWithData.intRectX : contourWithData.intRectX + contourWithData.intRectWidth]

    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))             # resize image, this will be more consistent for recognition and storage

    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))      # flatten image into 1d numpy array

    npaROIResized = np.float32(npaROIResized)       # convert from 1d numpy array of ints to 1d numpy array of floats

    retval, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized, k = 1)     # call KNN function find_nearest

    strCurrentChar = str(chr(int(npaResults[0][0])))                                             # get character from results

    strFinalString = strFinalString + strCurrentChar            # append current char to full string

In [19]:
print("\n" + strFinalString + "\n")                  # show the full string

cv2.imshow("imgTestingNumbers", img)      # show input image with green boxes drawn around found digits
cv2.waitKey(0)                                          # wait for user key press

cv2.destroyAllWindows()             # remove windows from memory


PIPN4J7579I

